In [161]:
# Project V2.0
# Contents : Seoul Restaurant csv file Process
# Coding   : ejlee
# Date     : 2023.05.19
# Description : 서울시 관광식당 인허가 정보.csv -> tour_r.csv 각각 읽어서 합치기

import numpy as np
import pandas as pd
import datetime


df_source = pd.read_csv('./srData/01tour_r.csv', encoding='CP949')

#필요컬럼 필수-이름 선택-우편번호?, 지번주소, 도로명주소, 전화번호 => 새로운 CSV로 저장  
df_data = pd.DataFrame({
    'store': df_source.loc[df_source['영업상태코드'] == 1, '사업장명'].values,
    'tel': df_source.loc[df_source['영업상태코드'] == 1, '전화번호'].values,
    'jibeon addr': df_source.loc[df_source['영업상태코드'] == 1, '지번주소'].values,
    'road addr': df_source.loc[df_source['영업상태코드'] == 1, '도로명주소'].values,
    'gu': 'nan' ,
    'main food': 'nan', #주된음식
    'reg date': datetime.datetime.now().strftime("%Y-%m-%d")
})

#구 : 도로명주소에서 구 정보만 추가
#df_data['gu'] = df_data['road addr'].apply(lambda x: x.split(' ')[1] if len(x.split(' ')) > 1 else '')
df_data['gu'] = df_data['road addr'].apply(lambda x: x.split(' ')[1] if isinstance(x, str) and len(x.split(' ')) > 1 else '')

df_data.head() 

#csv파일 저장
df_data.to_csv('./svData/restaurant.csv', index = False, encoding = 'CP949') #서울시 관광식당 인허가 정보.csv

In [162]:
df_svc = pd.read_csv('./svData/restaurant.csv', encoding = 'CP949') #135건

df_svc['main food'] = df_svc['main food'].astype(str)
df_svc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135 entries, 0 to 134
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   store        135 non-null    object
 1   tel          123 non-null    object
 2   jibeon addr  135 non-null    object
 3   road addr    121 non-null    object
 4   gu           121 non-null    object
 5   main food    135 non-null    object
 6   reg date     135 non-null    object
dtypes: object(7)
memory usage: 7.5+ KB


In [164]:
df_source1 = pd.read_csv('./srData/02sd.csv', encoding = 'CP949') #서울시 서대문구 모범음식점 지정 현황.csv 167건

#붙일 데이터 컬럼 추가하고 그외 컬럼 삭제하기
df_source1['store'] = df_source1['업소명']
df_source1['tel']    = np.nan
df_source1['jibeon addr'] = df_source1['소재지지번']
df_source1['road addr'] = df_source1['소재지도로명']
df_source1['gu'] = df_source1['소재지도로명'].apply(lambda x: x.split(' ')[1] if isinstance(x, str) and len(x.split(' ')) > 1 else '')
df_source1['main food'] = df_source1['주된음식']
df_source1['reg date'] = datetime.datetime.now().strftime("%Y-%m-%d")

df_source1 = df_source1.drop(df_source1.columns.difference(['store','tel','jibeon addr','road addr','gu','main food','reg date']), axis = 1)

# 열로 데이터프레임 붙이기
merged_df = pd.concat([df_svc, df_source1], ignore_index=True)

#csv파일 저장
merged_df.to_csv('./svData/restaurant.csv', index = False, encoding = 'CP949') #서울시 관광식당 인허가 정보.csv

merged_df.info()
merged_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302 entries, 0 to 301
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   store        302 non-null    object
 1   tel          123 non-null    object
 2   jibeon addr  302 non-null    object
 3   road addr    286 non-null    object
 4   gu           288 non-null    object
 5   main food    294 non-null    object
 6   reg date     302 non-null    object
dtypes: object(7)
memory usage: 16.6+ KB


,store,tel,jibeon addr,road addr,gu,main food,reg date
0,(주)아모제마르쉐신촌점,322-7757,서울특별시 서대문구 창천동 72-14번지 캠퍼스프라자 지1층,"서울특별시 서대문구 신촌로 73 (창천동,캠퍼스프라자 지1층)",서대문구,nan,2023-05-20
1,롸이즈온(주)베니건스이대점,NaN,서울특별시 서대문구 대현동 101-7번지,서울특별시 서대문구 신촌역로 10 (대현동),서대문구,nan,2023-05-20
2,씨제이푸드(주) 에프터더레인 안국점 (정진구),02-730-2051,서울특별시 종로구 화동 117번지,서울특별시 종로구 북촌로 31-22 (화동),종로구,nan,2023-05-20
3,안나푸르나(Anna Purna),747-2976,서울특별시 종로구 창신동 695-1번지 3층,"서울특별시 종로구 종로 291 (창신동,3층)",종로구,nan,2023-05-20
4,새골목집,749-3336,서울특별시 용산구 이태원동 116-15번지 지상1층,"서울특별시 용산구 이태원로27가길 50 (이태원동,지상1층)",용산구,nan,2023-05-20


In [167]:
df = pd.read_csv('./svData/restaurant.csv', encoding = 'CP949')
df.info()

df['gu'] = df['jibeon addr'].apply(lambda x: x.split(' ')[1] if isinstance(x, str) and len(x.split(' ')) > 1 else '')
df['gu'].isnull().sum() #16

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302 entries, 0 to 301
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   store        302 non-null    object
 1   tel          123 non-null    object
 2   jibeon addr  302 non-null    object
 3   road addr    286 non-null    object
 4   gu           286 non-null    object
 5   main food    159 non-null    object
 6   reg date     302 non-null    object
dtypes: object(7)
memory usage: 16.6+ KB


array(['서대문구', '종로구', '용산구', nan, '동대문구', '성북구', '노원구', '양천구', '구로구',
       '관악구', '강남구'], dtype=object)

In [283]:
df['gu'].value_counts() #sort=True 한글 안됨

서대문구    169
용산구      49
강남구      43
구로구      11
성북구      10
동대문구      6
관악구       6
노원구       3
양천구       3
종로구       2
Name: gu, dtype: int64